In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
using NLsolve
using LinearAlgebra
include("9msctrl.jl")
include("Initial condition variation.jl")
include("Perturbation time.jl")
include("Contour Plots.jl")
import LinearAlgebra: norm
using Plotly

┌ Info: Recompiling stale cache file C:\Users\Kaijie\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.
┌ Info: Recompiling stale cache file C:\Users\Kaijie\.julia\compiled\v1.2\NLsolve\KFCNP.ji for NLsolve [2774e3e8-f4cf-5e23-947b-6d7e65073b56]
└ @ Base loading.jl:1240


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-3697446873955285771\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15805715192909379232\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-3697446873955285771\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
Re = 100

Lx = 4π
Lz = 2π

cα = 2π/Lx
cβ = π/2
cγ = 2π/Lz
cβcβ = cβ*cβ
cβcγ = cβ*cγ
cαcα = cα*cα
cγcγ = cγ*cγ
cαcβcγ = cα*cβcγ
sqrt23 = sqrt(2/3)
sqrt32 = sqrt(3/2)
sqrt6 = sqrt(6)
sqrtcβcβpluscγcγ = sqrt(cβcβ + cγcγ)
sqrtcαcαpluscγcγ = sqrt(cαcα + cγcγ)
sqrtcαcαpluscβcβpluscγcγ = sqrt(cαcα + cβcβ + cγcγ)

1.9280563011157998

In [3]:
function f!(F, u)
    F[1] = cβcβ/Re - (cβcβ*u[1])/Re + (sqrt32*cβcγ*u[2]*u[3])/sqrtcβcβpluscγcγ- (sqrt32*cβcγ*u[6]*u[8])/sqrtcαcαpluscβcβpluscγcγ
    F[2] = -((((4*cβcβ)/3 + cγcγ)*u[2])/Re) + (5*sqrt23*cγcγ*u[4]*u[6])/(3*sqrtcαcαpluscγcγ) - (cγcγ*u[5]*u[7])/(sqrt6*sqrtcαcαpluscγcγ) - (cαcβcγ*u[5]*u[8])/(sqrt6*sqrtcαcαpluscγcγ*sqrtcαcαpluscβcβpluscγcγ) - (sqrt32*cβcγ*(u[1]*u[3] + u[3]*u[9]))/sqrtcβcβpluscγcγ
    F[3] = -(((cβcβ + cγcγ)*u[3])/Re) + (sqrt23*cαcβcγ*(u[5]*u[6] + u[4]*u[7]))/(sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ) + ((-3*cγcγ*(cαcα + cγcγ) + cβcβ*(3*cαcα + cγcγ))*u[4]*u[8])/(sqrt6*sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ*sqrtcαcαpluscβcβpluscγcγ)
    F[4] = -((3*cαcα + 4*cβcβ)*u[4])/(3*Re) - (cα*u[1]*u[5])/sqrt6 - (5*sqrt23*cαcα*u[2]*u[6])/(3*sqrtcαcαpluscγcγ) - (sqrt32*cαcβcγ*u[3]*u[7])/(sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ) - (sqrt32*cαcα*cβcβ*u[3]*u[8])/(sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ*sqrtcαcαpluscβcβpluscγcγ) - (cα*u[5]*u[9])/sqrt6
    F[5] = (cα*u[1]*u[4])/sqrt6 - ((cαcα + cβcβ)*u[5])/Re + (sqrt23*cαcβcγ*u[3]*u[6])/(sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ) +    (cαcα*u[2]*u[7])/(sqrt6*sqrtcαcαpluscγcγ) - (cαcβcγ*u[2]*u[8])/(sqrt6*sqrtcαcαpluscγcγ*sqrtcαcαpluscβcβpluscγcγ) + (cα*u[4]*u[9])/sqrt6
    F[6] = (5*sqrt23*(cαcα - cγcγ)*u[2]*u[4])/(3*sqrtcαcαpluscγcγ) - (2*sqrt23*cαcβcγ*u[3]*u[5])/(sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ) - ((3*cαcα + 4*cβcβ + 3*cγcγ)*u[6])/(3*Re) + (cα*u[1]*u[7])/sqrt6 + (sqrt32*cβcγ*u[1]*u[8])/sqrtcαcαpluscβcβpluscγcγ + (cα*u[7]*u[9])/sqrt6 + (sqrt32*cβcγ*u[8]*u[9])/sqrtcαcαpluscβcβpluscγcγ
    F[7] = (cαcβcγ*u[3]*u[4])/(sqrt6*sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ) + ((-cαcα + cγcγ)*u[2]*u[5])/(sqrt6*sqrtcαcαpluscγcγ) - (cα*u[1]*u[6])/sqrt6 - ((cαcα + cβcβ + cγcγ)*u[7])/Re - (cα*u[6]*u[9])/sqrt6
    F[8] = (cγcγ*(3*cαcα - cβcβ + 3*cγcγ)*u[3]*u[4])/(sqrt6*sqrtcαcαpluscγcγ*sqrtcβcβpluscγcγ*sqrtcαcαpluscβcβpluscγcγ) + (sqrt23*cαcβcγ*u[2]*u[5])/(sqrtcαcαpluscγcγ*sqrtcαcαpluscβcβpluscγcγ) - ((cαcα + cβcβ + cγcγ)*u[8])/Re
    F[9] = (sqrt32*cβcγ*u[2]*u[3])/sqrtcβcβpluscγcγ - (sqrt32*cβcγ*u[6]*u[8])/sqrtcαcαpluscβcβpluscγcγ - (9*cβcβ*u[9])/Re
end

f! (generic function with 1 method)

In [4]:
function findpoints(Re)
    Re = Re
    z = zeros(9,9)
    for i in 1:9
        dummy = zeros(9)
        dummy[i] = Float64(1)
        answer = nlsolve(f!, dummy)
        z[:,i] = answer.zero
    end
    return z
end

findpoints (generic function with 1 method)

In [5]:
# function f!(F, x)
#     F[1] = (x[1]+3)*(x[2]^3-7)+18
#     F[2] = sin(x[2]*exp(x[1])-1)
# end

# function j!(J, x)
#     J[1, 1] = x[2]^3-7
#     J[1, 2] = 3*x[2]^2*(x[1]+3)
#     u = exp(x[1])*cos(x[2]*exp(x[1])-1)
#     J[2, 1] = x[2]*u
#     J[2, 2] = u
# end

# z = nlsolve(f!, j!, [ 0.1; 1.2])
nlsolve(f!, ones(9)) #equilibrium points always 1.0, remaining 0

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
 * Zero: [1.0000001378681795, -2.669681594725821e-7, 9.511695832779333e-9, -4.651207791049009e-9, 4.783219031838007e-9, 4.4448892550006294e-8, -2.000330681405362e-7, 3.770863117434579e-8, 1.5318686712572296e-8]
 * Inf-norm of residuals: 0.000000
 * Iterations: 287
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 287
 * Jacobian Calls (df/dx): 213

In [9]:
#at equilibrium point (1,0,0,0,0,0,0,0,0), the Jacobian:
Jacobian  = zeros(9,9)
Jacobian[1,1] = -(cβ)^2/Re
Jacobian[2,2] = -(4*cβ^2/3 + cγ^2)/Re
Jacobian[2,3] = -sqrt32*cβcγ/sqrtcβcβpluscγcγ
Jacobian[3,3] = -(cβ^2 + cγ^2)/Re
Jacobian[4,4] = -(3*cα^2 + 4*cβ^2)/(3*Re)
Jacobian[4,5] = -cα/sqrt6
Jacobian[5,4] = cα/sqrt6
Jacobian[5,5] = -(cα^2 + cβ^2)/Re
Jacobian[6,7] = cα/sqrt6
Jacobian[6,6] = -(3*cα^2 + 4*cβ^2 + 3*cγ^2)/(3*Re)
Jacobian[6,8] = sqrt32*cβcγ/sqrtcβcβpluscγcγ
Jacobian[7,6] = -cα/sqrt6;
Jacobian[7,7] = -(cα^2 + cβ^2 + cγ^2)/Re
Jacobian[8,8] = -(cα^2 + cβ^2 + cγ^2)/Re
Jacobian[9,9] = (-9*cβ^2)/Re

-0.22206609902451058

In [10]:
eigvals(Jacobian)

9-element Array{Complex{Float64},1}:
  -0.22206609902451058 + 0.0im                
 -0.042898681336964534 + 0.0im                
  -0.04128634616984395 - 0.2040827169706929im 
  -0.04128634616984395 + 0.2040827169706929im 
   -0.0371740110027234 + 0.0im                
 -0.034674011002723396 + 0.0im                
  -0.03128634616984398 - 0.20408271697069288im
  -0.03128634616984398 + 0.20408271697069288im
 -0.024674011002723394 + 0.0im                

In [11]:
eigvecs(Jacobian)

9×9 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im  …        0.0+0.0im       1.0+0.0im
 0.0+0.0im  1.0+0.0im           0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.0+0.0im           0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.0+0.0im     -0.707107+0.0im       0.0+0.0im
 0.0+0.0im  0.0+0.0im     0.0142455+0.706963im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  …        0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.0+0.0im           0.0+0.0im       0.0+0.0im
 0.0+0.0im  0.0+0.0im           0.0+0.0im       0.0+0.0im
 1.0+0.0im  0.0+0.0im           0.0+0.0im       0.0+0.0im